In [2812]:
import pandas as pd
import openpyxl
from datetime import datetime
import numpy as np
from dateutil.relativedelta import relativedelta
from scipy.stats import describe, ttest_1samp
from pathlib import Path
from openpyxl import load_workbook
import os

## Read the data

In [2813]:
# rfr file, rm file
file_name_rfr = r"..\Refinitiv\New rfr.xlsx"
file_name_rm = r"..\Refinitiv\New rm.xlsx"

#get the sorted fin file
# file_name_str = r"..\Refinitiv\Analysis\AU\AU_Refinitiv_fin_data_portfolio.xlsx"
# file_name_str = r"..\Refinitiv\Analysis\CN\CN_Refinitiv_fin_data_portfolio.xlsx"
# file_name_str = r"..\Refinitiv\Analysis\HK\HK_Refinitiv_fin_data_portfolio.xlsx"
# file_name_str = r"..\Refinitiv\Analysis\MY\MY_Refinitiv_fin_data_portfolio.xlsx"
file_name_str = r"..\Refinitiv\Analysis\SG\SG_Refinitiv_fin_data_portfolio.xlsx"



# indicator = 'e'
# indicator = 's'
# indicator = 'g'
# indicator = 'esg'
# indicator = 'esg_value'
# indicator = 'value'
# indicator = 'mv'
# indicator = 'roa'
indicator = 'vol_180d'
## indicator = 'return_com_eqy'

file_name = Path(file_name_str).stem
fin_file = pd.ExcelFile(file_name_str)
rfr_file = pd.ExcelFile(file_name_rfr)
rm_file = pd.ExcelFile(file_name_rm)

findata_sheet = pd.read_excel(
    fin_file, sheet_name="main", index_col=0, header=0)

porfolio_selection = pd.DataFrame(findata_sheet)


porfolio_selection.head()


,firm_name,tkr,sec_no,year,esg,e,s,g,es,eg,...,sp_z,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value
firm_no,,,,,,,,,,,,,,,,,,,,,
14,China Aviation Oil (Singapore) Corporation Ltd,CNAO.SI,1,2017,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.707107,11.568620,12.017363,0.634619,-0.707107,-7.071068e-01,10.801095,7.893590,-0.707107,NaN
14,China Aviation Oil (Singapore) Corporation Ltd,CNAO.SI,1,2018,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.707107,NaN,24.357731,NaN,NaN,0.000000e+00,6.932902,2.354634,-0.707107,NaN
14,China Aviation Oil (Singapore) Corporation Ltd,CNAO.SI,1,2019,18.956321,5.950305,18.583850,32.960784,110.579577,196.126724,...,-0.707107,3.855826,8.784869,6.970720,-0.707107,-2.357023e-01,4.419383,1.756878,0.707107,9.360309
14,China Aviation Oil (Singapore) Corporation Ltd,CNAO.SI,1,2020,20.163526,7.326079,21.149379,31.933677,154.942011,233.948625,...,-0.707107,NaN,8.362799,NaN,NaN,-1.110223e-16,2.870669,0.166325,0.707107,10.081763
14,China Aviation Oil (Singapore) Corporation Ltd,CNAO.SI,1,2021,25.214499,13.774284,34.274059,23.295721,472.100623,320.881872,...,-0.707107,NaN,1.798649,NaN,NaN,-7.071068e-01,9.659823,10.327521,-0.707107,12.253696


## Sorting the data by firm name and then date

In [2814]:
portfolio_selection_sorted = porfolio_selection.sort_values(
    by=['firm_name', 'year'])
portfolio_selection_sorted.head(10)

,firm_name,tkr,sec_no,year,esg,e,s,g,es,eg,...,sp_z,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value
firm_no,,,,,,,,,,,,,,,,,,,,,
1,AEM Holdings Ltd,AEM.SI,8,2017,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.459130,13.981730,16.324743,15.456103,-0.151591,-0.210635,27.389168,37.572848,0.308960,NaN
1,AEM Holdings Ltd,AEM.SI,8,2018,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.528780,5.940570,18.705622,18.444906,-0.692064,0.064237,14.126625,8.400372,1.469059,NaN
1,AEM Holdings Ltd,AEM.SI,8,2019,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.086577,7.073184,13.736628,14.561529,-0.457606,-0.423594,17.446831,10.026366,1.148245,NaN
1,AEM Holdings Ltd,AEM.SI,8,2020,27.381327,23.196373,34.181488,21.183007,792.886556,491.368931,...,0.166205,2.581016,17.494781,18.753909,-0.795235,-0.006628,16.179751,12.640899,1.460142,13.687349
1,AEM Holdings Ltd,AEM.SI,8,2021,30.768458,24.611775,32.681267,35.000344,804.343977,861.420566,...,0.844262,12.238264,20.671748,18.021131,-0.467978,-0.242646,10.140624,5.701218,1.309775,15.262906
1,AEM Holdings Ltd,AEM.SI,8,2022,39.545171,29.218572,43.867555,44.805155,1281.747315,1309.142638,...,-0.652048,3.319671,20.190899,15.184893,-1.111053,-0.206321,13.202667,3.405021,1.054739,19.669425
4,Bumitama Agri Ltd,BUMI.SI,5,2017,NaN,NaN,NaN,NaN,NaN,NaN,...,0.005827,11.367127,13.154107,9.625199,-0.185656,-0.034290,10.367825,9.202317,-0.096828,NaN
4,Bumitama Agri Ltd,BUMI.SI,5,2018,NaN,NaN,NaN,NaN,NaN,NaN,...,0.539603,15.798456,13.464667,6.888307,0.338804,0.199227,5.898729,4.708877,0.475866,NaN
4,Bumitama Agri Ltd,BUMI.SI,5,2019,51.540230,50.917832,48.398559,57.652977,2464.349685,2935.564611,...,0.057587,10.355942,9.264575,6.499889,0.167906,0.121002,5.969361,3.727632,-0.274625,25.830616


## Ranking stocks percentile
| Country/Region | Percentile chosen (Top(%))|
| --- | --- |
| Australia (AU)| 20 |
| China (CN)| 20 |
| Malaysia (SG)| 20 |
| Hong Kong (SG)| 25 |
| Singapore (SG)| 25 |

## Excess Return
At the end of each month, compute the cross-sectional portfolio return of esg_bottom20. For example, at the end of August, after selecting the bottom 20% stocks based on August ESG scores, compute the raw returns of each stocks in the high-rated portfolio (the bottom 20%) by the following equation: $$ret_{i}  = P_{1}/P_{0} -1 $$(for example, after ranking stocks at the end of Aug, calculate the individual returns by using $$(Sept-price/Aug-price) – 1)$$ Next, compute excess returns of each stocks: $$r_{i} = ret_{i} – r_{f}$$ where r_i is the excess return of stock i and r_f is the respective country’s monthly risk-free rate. So now we have r_1, …, r_n where n = number of stocks in the high-rated portfolio. Next, we equal-weight (average) all the r in the esg_bottom 20 and find the portfolio mean (denoted as μ_t) for each month. This generates the High portfolio excess return (equal weight) in the Excel file for each month. Scroll down the Excel file, compute the mean of all the months’ excess returns, and test for significance (H0: mean > 0). Indicate the t-stat value.

In [2815]:
"""
change data format
"""

# portfolio_selection_sorted['date'] = portfolio_selection_sorted['date'].astype(
#     'datetime64')
# portfolio_selection_sorted.dtypes


'\nchange data format\n'

In [2816]:
# new_companies_data.to_excel("SG_Bloomberg_fin_data_portfolio.xlsx", sheet_name="main")
quantile_rank = 0
quantile_path = ''
quantile_folder = ''


quantile_folder = "..\Refinitiv\Analysis_bottom\\" + file_name[:2] + "\\" + indicator
final_quantile_folder = "..\Refinitiv\Analysis\\" + file_name[:2] + "\\" + indicator

quantile_path = final_quantile_folder + "\\" + file_name[:2] +"_portfolio_" + indicator + "_quantile_rank.xlsx"
bottom_path = final_quantile_folder + "\\" + file_name[:2] + "_" + indicator + "_bottom20.xlsx"
reporting_path = quantile_folder + "\\" + file_name[:2] + "_" + indicator + "_int_reporting_low.xlsx"


rfr_sheet = pd.read_excel(rfr_file, sheet_name= file_name[:2]+"_rfr" )
rm_sheet = pd.read_excel(rm_file, sheet_name= file_name[:2]+"_rm" )

try:
    os.makedirs(quantile_folder)
    os.makedirs(final_quantile_folder)
except FileExistsError:
    # directory already exists
    pass

if file_name[:2].lower() == 'AU'.lower():
    quantile_rank = 5

elif file_name[:2].lower() == 'CN'.lower():
    quantile_rank = 5

elif file_name[:2].lower() == 'HK'.lower():
    quantile_rank = 5

elif file_name[:2].lower() == 'MY'.lower():
    quantile_rank = 4

elif file_name[:2].lower() == 'SG'.lower():
    quantile_rank = 4
    

In [2817]:
rfr_sheet = rfr_sheet.rename({'Year': 'year'.lower(), file_name[:2]+'_rfr_yr': 'rfr'}, axis=1)
rfr_sheet

,year,rfr
0,2017,0.017620
1,2018,0.020130
2,2019,0.016570
3,2020,0.003050
4,2021,0.004420
5,mean_yr,0.012358


In [2818]:
rm_sheet = rm_sheet.rename({'Year': 'year'.lower(), file_name[:2]+'_rm_yr': 'rm'}, axis=1)
rm_sheet

,year,rm
0,2017,0.116881
1,2018,-0.131644
2,2019,0.010238
3,2020,-0.098272
4,2021,0.108782
5,mean_yr,0.001197


In [2819]:
rfr_rm_sheet = pd.merge(rfr_sheet, rm_sheet, how='left', on='year')
rfr_rm_sheet

,year,rfr,rm
0,2017,0.017620,0.116881
1,2018,0.020130,-0.131644
2,2019,0.016570,0.010238
3,2020,0.003050,-0.098272
4,2021,0.004420,0.108782
5,mean_yr,0.012358,0.001197


In [2820]:
"""
gets all the companies, dropping the empty columns
"""
companies = portfolio_selection_sorted.iloc[:, 0].drop_duplicates()

#set the data to a list
companies = companies.tolist()


#ensure the number of companies exist
len(companies)


52

In [2821]:
# new_companies_data.to_excel("SG_data_2.28_esgj3mean.xlsx")

### Rank the companies 

In [2822]:
companies_sray = []
all_quantile_rank = []

initial_year = 2017
stop_year = 2023


for year in range(initial_year, stop_year+1, 1):

        # temp_dataframe = fin_data.loc[(fin_data['sec_no']==k+1) & (fin_data['date']>=start_date) & (fin_data['date']<=end_date)].copy()
        temp_dataframe = portfolio_selection_sorted.loc[(
            portfolio_selection_sorted['year'] == year)].copy()

        
        """
        sort by the relative value and then divide them to relative division
        """

        temp_dataframe.sort_values(by=[indicator], inplace=True)

        temp_dataframe['quantile_rank'] = pd.qcut(temp_dataframe[indicator],
                                                    q=quantile_rank, labels=False, duplicates='drop')

        temp_dataframe_quantile = temp_dataframe.copy()

        companies_sray.append(temp_dataframe)

        all_quantile_rank.append(temp_dataframe_quantile)


In [2823]:
all_sort_companies_data = pd.concat(all_quantile_rank)

all_sort_companies_data.to_excel(
    quantile_path, sheet_name="main")


In [2824]:
sort_companies_data = pd.concat(companies_sray)
sort_companies_data.head()

,firm_name,tkr,sec_no,year,esg,e,s,g,es,eg,...,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank
firm_no,,,,,,,,,,,,,,,,,,,,,
41,Hotel Properties Ltd,HPPS.SI,4,2017,NaN,NaN,NaN,NaN,NaN,NaN,...,1.693572,6.853578,4.470617,-1.154204,-0.309949,4.979294,1.932443,0.329775,NaN,0
32,Great Eastern Holdings Ltd,GELA.SI,7,2017,32.771085,29.060755,34.766197,38.478261,1010.331945,1118.207325,...,0.093682,1.739115,1.428202,-1.152101,-0.966200,3.402568,5.755630,-0.353976,15.902443,0
38,Hong Leong Finance Ltd,HLSF.SI,7,2017,NaN,NaN,NaN,NaN,NaN,NaN,...,2.466058,1.739115,1.428202,0.508992,0.619949,3.402568,5.755630,-0.471385,NaN,0
71,Sheng Siong Group Ltd,SHEN.SI,5,2017,NaN,NaN,NaN,NaN,NaN,NaN,...,18.327547,13.154107,9.625199,0.537489,0.101688,10.367825,9.202317,0.782990,NaN,0
73,Singapore Airlines Ltd,SIAL.SI,3,2017,55.639475,59.621212,59.838752,45.077295,3567.658939,2687.562948,...,0.681292,9.300856,14.976229,-0.575550,-0.502838,5.883379,5.247748,-0.774353,27.568318,0


In [2825]:
all_sort_companies_data.columns

Index(['firm_name', 'tkr', 'sec_no', 'year', 'esg', 'e', 's', 'g', 'es', 'eg',
       'sg', 'esg_cubic', 'px', 'fcf', 'pe', 'beta_5y', 'beta_6m', 'roa',
       'vol_180d', 'mv', 'vol_90d', 'ret_ln', 'ep', 'ep_sec_mean', 'ep_sec_sd',
       'ep_z', 'sp', 'sp_sec_mean', 'sp_sec_sd', 'sp_z', 'cfp', 'cfp_sec_mean',
       'cfp_sec_sd', 'cfp_z', 'value', 'roa_sec_mean', 'roa_sec_sd', 'roa_z',
       'esg_value', 'quantile_rank'],
      dtype='object')

In [2826]:
all_sort_companies_data = all_sort_companies_data.reset_index()
all_sort_companies_data = pd.merge(all_sort_companies_data, rfr_rm_sheet, how='left', on='year')
all_sort_companies_data.insert(all_sort_companies_data.columns.get_loc('ret_ln')+1, 'rfr', all_sort_companies_data.pop('rfr'))
all_sort_companies_data.insert(all_sort_companies_data.columns.get_loc('rfr')+1, 'rm', all_sort_companies_data.pop('rm'))

all_sort_companies_data.head()


,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank
0,41,Hotel Properties Ltd,HPPS.SI,4,2017,NaN,NaN,NaN,NaN,NaN,...,1.693572,6.853578,4.470617,-1.154204,-0.309949,4.979294,1.932443,0.329775,NaN,0
1,32,Great Eastern Holdings Ltd,GELA.SI,7,2017,32.771085,29.060755,34.766197,38.478261,1010.331945,...,0.093682,1.739115,1.428202,-1.152101,-0.966200,3.402568,5.755630,-0.353976,15.902443,0
2,38,Hong Leong Finance Ltd,HLSF.SI,7,2017,NaN,NaN,NaN,NaN,NaN,...,2.466058,1.739115,1.428202,0.508992,0.619949,3.402568,5.755630,-0.471385,NaN,0
3,71,Sheng Siong Group Ltd,SHEN.SI,5,2017,NaN,NaN,NaN,NaN,NaN,...,18.327547,13.154107,9.625199,0.537489,0.101688,10.367825,9.202317,0.782990,NaN,0
4,73,Singapore Airlines Ltd,SIAL.SI,3,2017,55.639475,59.621212,59.838752,45.077295,3567.658939,...,0.681292,9.300856,14.976229,-0.575550,-0.502838,5.883379,5.247748,-0.774353,27.568318,0


In [2827]:
df_sec_company_count_sum = []
df_sec_company_count = pd.DataFrame(sort_companies_data.groupby(['quantile_rank','year'])['tkr'].count())
df_sec_company_count = df_sec_company_count.rename({'tkr':'tkr_count'}, axis=1)
df_sec_company_nunique = pd.DataFrame(sort_companies_data.groupby(['quantile_rank','year'])['tkr'].nunique())
df_sec_company_nunique = df_sec_company_nunique.rename({'tkr':'tkr_nunique'}, axis=1)

df_sec_company_count = pd.merge(df_sec_company_count, df_sec_company_nunique, on=['quantile_rank','year'])

In [2828]:
df_sec_company_count_sum = list()
# for rank in range (0, quantile_rank, 1):
groups = df_sec_company_count.groupby(level='quantile_rank')

    # loop through the groups to create separate dataframes
dfs = {}
for name, group in groups:
    dfs[name] = group.copy()
        
    # display the separate dataframes
for name, df in dfs.items():

        # Create a new row with the sum of all values above it
    sum_row = df.sum()
    sum_index = (name, 'Sum')
    sum_row.name = sum_index

        # Append the new row to the original DataFrame
    temp_1 = df.append(sum_row)
    df_sec_company_count_sum.append(temp_1)
# df_sec_company_count_all = pd.concat(df_sec_company_count_sum)
df_sec_company_count_sum_pd = pd.concat(df_sec_company_count_sum)

In [2829]:
book = load_workbook(quantile_path)
writer = pd.ExcelWriter(quantile_path, engine = 'openpyxl')
writer.book = book
df_sec_company_count_sum_pd.to_excel(writer, sheet_name='quantile sec no', index=True)
writer.close()

In [2830]:
# sort_companies_data['date'] = sort_companies_data['date'].dt.strftime(
#     '%Y-%m')

all_sort_companies_data = all_sort_companies_data.sort_values(
    by=["year", "quantile_rank"])

all_sort_companies_data.head()


c:\Users\heihe\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\displayhook.py:275: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank
0,41,Hotel Properties Ltd,HPPS.SI,4,2017,NaN,NaN,NaN,NaN,NaN,...,1.693572,6.853578,4.470617,-1.154204,-0.309949,4.979294,1.932443,0.329775,NaN,0
1,32,Great Eastern Holdings Ltd,GELA.SI,7,2017,32.771085,29.060755,34.766197,38.478261,1010.331945,...,0.093682,1.739115,1.428202,-1.152101,-0.966200,3.402568,5.755630,-0.353976,15.902443,0
2,38,Hong Leong Finance Ltd,HLSF.SI,7,2017,NaN,NaN,NaN,NaN,NaN,...,2.466058,1.739115,1.428202,0.508992,0.619949,3.402568,5.755630,-0.471385,NaN,0
3,71,Sheng Siong Group Ltd,SHEN.SI,5,2017,NaN,NaN,NaN,NaN,NaN,...,18.327547,13.154107,9.625199,0.537489,0.101688,10.367825,9.202317,0.782990,NaN,0
4,73,Singapore Airlines Ltd,SIAL.SI,3,2017,55.639475,59.621212,59.838752,45.077295,3567.658939,...,0.681292,9.300856,14.976229,-0.575550,-0.502838,5.883379,5.247748,-0.774353,27.568318,0


**SRAY- Ret already calculated using excel by Dr May**
<br>**BLOOMBERG- Ret calculated using PYTHON CODE 2_portfolio.ipynb**

In [2831]:

sort_companies_data_bottom20per = all_sort_companies_data.query("quantile_rank == 0")

sort_companies_data_bottom20per.head()


,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank
0,41,Hotel Properties Ltd,HPPS.SI,4,2017,NaN,NaN,NaN,NaN,NaN,...,1.693572,6.853578,4.470617,-1.154204,-0.309949,4.979294,1.932443,0.329775,NaN,0
1,32,Great Eastern Holdings Ltd,GELA.SI,7,2017,32.771085,29.060755,34.766197,38.478261,1010.331945,...,0.093682,1.739115,1.428202,-1.152101,-0.966200,3.402568,5.755630,-0.353976,15.902443,0
2,38,Hong Leong Finance Ltd,HLSF.SI,7,2017,NaN,NaN,NaN,NaN,NaN,...,2.466058,1.739115,1.428202,0.508992,0.619949,3.402568,5.755630,-0.471385,NaN,0
3,71,Sheng Siong Group Ltd,SHEN.SI,5,2017,NaN,NaN,NaN,NaN,NaN,...,18.327547,13.154107,9.625199,0.537489,0.101688,10.367825,9.202317,0.782990,NaN,0
4,73,Singapore Airlines Ltd,SIAL.SI,3,2017,55.639475,59.621212,59.838752,45.077295,3567.658939,...,0.681292,9.300856,14.976229,-0.575550,-0.502838,5.883379,5.247748,-0.774353,27.568318,0


In [2832]:
sort_companies_data_bottom20per.columns

Index(['firm_no', 'firm_name', 'tkr', 'sec_no', 'year', 'esg', 'e', 's', 'g',
       'es', 'eg', 'sg', 'esg_cubic', 'px', 'fcf', 'pe', 'beta_5y', 'beta_6m',
       'roa', 'vol_180d', 'mv', 'vol_90d', 'ret_ln', 'rfr', 'rm', 'ep',
       'ep_sec_mean', 'ep_sec_sd', 'ep_z', 'sp', 'sp_sec_mean', 'sp_sec_sd',
       'sp_z', 'cfp', 'cfp_sec_mean', 'cfp_sec_sd', 'cfp_z', 'value',
       'roa_sec_mean', 'roa_sec_sd', 'roa_z', 'esg_value', 'quantile_rank'],
      dtype='object')

In [2833]:
companies = portfolio_selection_sorted.iloc[:, 0].drop_duplicates().tolist()
print(companies)

['AEM Holdings Ltd', 'Bumitama Agri Ltd', 'CSE Global Ltd', 'China Aviation Oil (Singapore) Corporation Ltd', 'China Everbright Water Ltd', 'Comfortdelgro Corporation Ltd', 'Cosco Shipping International (Singapore) Co Ltd', 'DBS Group Holdings Ltd', 'DFI Retail Group Holdings Ltd', 'Delfi Ltd', 'First Resources Ltd', 'Fraser and Neave Ltd', 'Genting Singapore Ltd', 'Golden Agri-Resources Ltd', 'Golden Energy and Resources Ltd', 'Great Eastern Holdings Ltd', 'Haw Par Corporation Ltd', 'Hong Leong Finance Ltd', 'Hotel Properties Ltd', 'Hutchison Port Holdings Trust', 'Japfa Ltd', 'Jardine Cycle & Carriage Ltd', 'Jardine Matheson Holdings Ltd', 'Keppel Corporation Ltd', 'Keppel Infrastructure Trust', 'Mandarin Oriental International Ltd', 'NetLink NBN Trust', 'Oversea-Chinese Banking Corporation Ltd', 'Q & M Dental Group (Singapore) Ltd', 'Raffles Medical Group Ltd', 'Riverstone Holdings Ltd', 'SATS Ltd', 'SBS Transit Ltd', 'SIA Engineering Company Ltd', 'Sembcorp Industries Ltd', 'Sembco

In [2834]:
companies_sec=[]
initial_year=2017
stop_year=2023

company_dict1 = {}
company_dict_all = {}
sum_esg_sray = {}
sum_ret_dict = {}
# company_dict2 = {}

#loop year, loop every months to get the companies quantile rank =0 and save it into company dictionary 
for year in range(initial_year,stop_year+1,1):    

        each_year_companies = sort_companies_data_bottom20per.loc[(
            sort_companies_data_bottom20per['year'] == year)]

        each_year_companies_all_quantile = all_sort_companies_data.loc[(
            all_sort_companies_data['year'] == year)]

        sum = each_year_companies[indicator].sum()
        sum_esg_sray.setdefault(year, sum)

        sum_ret = each_year_companies['ret_ln'].sum()
        sum_ret_dict.setdefault(year, sum_ret)

        companies = each_year_companies.iloc[:, 0]
        companies= companies.tolist() 

        total_companies = (each_year_companies_all_quantile.iloc[:, 0]).tolist()


        for c in range (len(companies)):
            company_dict1.setdefault(year,[]).append(companies[c])
            #company_dict2.setdefault(start_date,[]).append(companies[c])
        for c_all in range(len(total_companies)):
            company_dict_all.setdefault(
                year, []).append(total_companies[c_all])    
        

#make sure the list is correct     
#print(company_dict)
# print(f"company_dict1: {company_dict1}")
print(len(company_dict1))
print(f"sum_esg_sray_dic: {sum_esg_sray}")
print(f"sum_ret_dict: {sum_ret_dict}")
# print(len(company_dict2))


6
sum_esg_sray_dic: {2017: 180.80939409684873, 2018: 180.80939409684873, 2019: 180.80939409684873, 2020: 180.80939409684873, 2021: 180.80939409684873, 2022: 119.58051882742991, 2023: 0.0}
sum_ret_dict: {2017: -0.9916754889463335, 2018: 0.7561438808844098, 2019: -0.38069140327262574, 2020: 0.7948512499243356, 2021: 1.2122439216858008, 2022: -11.700879418363362, 2023: 0.0}


If it is equal-weighted, compute the mean of the stocks returns in the portfolio. 
$μ_(p,t)=  (\sum ret_{(i,t)} )/n$; i = 1, 2, 3, …, n
Where $ret_(i,t) = return-of-stock-i-at-time-t$, n = number of stocks in the portfolio


In [2835]:
no_of_companies = {}
equal_weight_dict = {}
for date_range, company_names in company_dict1.items():

    # date = datetime.strptime(date_range, '%Y-%m')

    #equal weightage
    # equal_weight = np.repeat(1/len(v), len(v))
    equal_weight = 1/len(company_names)
        # print(f"factor_weight: {factor_weight}")

    equal_weight_dict.setdefault(date_range, float(equal_weight))
    no_of_companies.setdefault(date_range, float(len(company_names)))

    # factor_weight_dict.setdefault(date_range, factor_weight)
print(f"equal_weight: {equal_weight_dict}")
print(f"no_of_companies: {no_of_companies}")


equal_weight: {2017: 0.07692307692307693, 2018: 0.07692307692307693, 2019: 0.07692307692307693, 2020: 0.07692307692307693, 2021: 0.07692307692307693, 2022: 0.1111111111111111}
no_of_companies: {2017: 13.0, 2018: 13.0, 2019: 13.0, 2020: 13.0, 2021: 13.0, 2022: 9.0}


### total companies from all quantile

In [2836]:
no_of_companies_all = {}
for date_range, company_names in company_dict_all.items():

    # date = datetime.strptime(date_range, '%Y-%m')

    no_of_companies_all.setdefault(date_range, float(len(company_names)))

    # factor_weight_dict.setdefault(date_range, factor_weight)
print(f"no_of_companies: {no_of_companies_all}")


no_of_companies: {2017: 52.0, 2018: 52.0, 2019: 52.0, 2020: 52.0, 2021: 52.0, 2022: 34.0}


## Calculate the number of companies (Bottom 20%) based on the date

In [2837]:
no_of_companies_items = no_of_companies.items()
no_of_companies_list = list(no_of_companies_items)

no_of_companies_df = pd.DataFrame(no_of_companies_list)
no_of_companies_df.columns = ['year', 'no of firms(bottom 20)']
no_of_companies_df.head()

,year,no of firms(bottom 20)
0,2017,13.0
1,2018,13.0
2,2019,13.0
3,2020,13.0
4,2021,13.0


## Calculate the number of companies based on the date

In [2838]:
no_of_companies_all_items = no_of_companies_all.items()
no_of_companies_all_list = list(no_of_companies_all_items)

no_of_companies_all_df = pd.DataFrame(no_of_companies_all_list)
no_of_companies_all_df.columns = ['year', 'no of firms']
no_of_companies_all_df.head()

,year,no of firms
0,2017,52.0
1,2018,52.0
2,2019,52.0
3,2020,52.0
4,2021,52.0


### Make the equal weight item into dataframe
Purpose: use for merging later on

In [2839]:
equal_weight_items = equal_weight_dict.items()
equal_weight_list = list(equal_weight_items)

equal_weight_df = pd.DataFrame(equal_weight_list)
equal_weight_df.columns = ['year', 'eq_wg']
equal_weight_df.head()


,year,eq_wg
0,2017,0.076923
1,2018,0.076923
2,2019,0.076923
3,2020,0.076923
4,2021,0.076923


In [2840]:
# no_of_companies_df.to_excel("eq_wg_no_of_companies_bottom20.xlsx")

In [2841]:
sum_ret_items = sum_ret_dict.items()
sum_ret_list = list(sum_ret_items)

sum_ret_df = pd.DataFrame(sum_ret_list)
sum_ret_df.columns = ['year', 'sum_ret']
sum_ret_df.replace(0, np.nan, inplace=True)
sum_ret_df.head()


,year,sum_ret
0,2017,-0.991675
1,2018,0.756144
2,2019,-0.380691
3,2020,0.794851
4,2021,1.212244


### Make the sum of the esg_sray into dataframe
Purpose: merge into complete dataframe to make the data to be able to vefiry

In [2842]:
sum_esg_sray_dic_items = sum_esg_sray.items()
sum_esg_sray_list = list(sum_esg_sray_dic_items)

sum_esg_sray_df = pd.DataFrame(sum_esg_sray_list)
sum_esg_sray_df.columns = ['year', 'sum_'+indicator]
sum_esg_sray_df.head()


,year,sum_vol_180d
0,2017,180.809394
1,2018,180.809394
2,2019,180.809394
3,2020,180.809394
4,2021,180.809394


### Merging with the data of top 20% **only**

In [2843]:
sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    equal_weight_df, on='year')
sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    sum_esg_sray_df, on='year')
sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    sum_ret_df, on='year')
sort_companies_data_bottom20per.head()


,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank,eq_wg,sum_vol_180d,sum_ret
0,41,Hotel Properties Ltd,HPPS.SI,4,2017,NaN,NaN,NaN,NaN,NaN,...,-1.154204,-0.309949,4.979294,1.932443,0.329775,NaN,0,0.076923,180.809394,-0.991675
1,32,Great Eastern Holdings Ltd,GELA.SI,7,2017,32.771085,29.060755,34.766197,38.478261,1010.331945,...,-1.152101,-0.966200,3.402568,5.755630,-0.353976,15.902443,0,0.076923,180.809394,-0.991675
2,38,Hong Leong Finance Ltd,HLSF.SI,7,2017,NaN,NaN,NaN,NaN,NaN,...,0.508992,0.619949,3.402568,5.755630,-0.471385,NaN,0,0.076923,180.809394,-0.991675
3,71,Sheng Siong Group Ltd,SHEN.SI,5,2017,NaN,NaN,NaN,NaN,NaN,...,0.537489,0.101688,10.367825,9.202317,0.782990,NaN,0,0.076923,180.809394,-0.991675
4,73,Singapore Airlines Ltd,SIAL.SI,3,2017,55.639475,59.621212,59.838752,45.077295,3567.658939,...,-0.575550,-0.502838,5.883379,5.247748,-0.774353,27.568318,0,0.076923,180.809394,-0.991675


### Calculate the factor weight

In [2844]:
if sort_companies_data_bottom20per[indicator].empty and sort_companies_data_bottom20per['sum_'+indicator].empty:
    sort_companies_data_bottom20per['fc_wg'] = None
else:
    sort_companies_data_bottom20per['fc_wg'] = sort_companies_data_bottom20per[indicator] / \
        sort_companies_data_bottom20per['sum_'+indicator]

sort_companies_data_bottom20per.head()


,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank,eq_wg,sum_vol_180d,sum_ret,fc_wg
0,41,Hotel Properties Ltd,HPPS.SI,4,2017,NaN,NaN,NaN,NaN,NaN,...,-0.309949,4.979294,1.932443,0.329775,NaN,0,0.076923,180.809394,-0.991675,0.048188
1,32,Great Eastern Holdings Ltd,GELA.SI,7,2017,32.771085,29.060755,34.766197,38.478261,1010.331945,...,-0.966200,3.402568,5.755630,-0.353976,15.902443,0,0.076923,180.809394,-0.991675,0.050378
2,38,Hong Leong Finance Ltd,HLSF.SI,7,2017,NaN,NaN,NaN,NaN,NaN,...,0.619949,3.402568,5.755630,-0.471385,NaN,0,0.076923,180.809394,-0.991675,0.062138
3,71,Sheng Siong Group Ltd,SHEN.SI,5,2017,NaN,NaN,NaN,NaN,NaN,...,0.101688,10.367825,9.202317,0.782990,NaN,0,0.076923,180.809394,-0.991675,0.071226
4,73,Singapore Airlines Ltd,SIAL.SI,3,2017,55.639475,59.621212,59.838752,45.077295,3567.658939,...,-0.502838,5.883379,5.247748,-0.774353,27.568318,0,0.076923,180.809394,-0.991675,0.071226


In [2845]:
sort_companies_data_bottom20per['r'] = (
    sort_companies_data_bottom20per['ret_ln'] - sort_companies_data_bottom20per['rfr'])

"""
r = ret -rfr
"""
sort_companies_data_bottom20per['eq_wg_r'] = (
    sort_companies_data_bottom20per['r'] * sort_companies_data_bottom20per['eq_wg'])
sort_companies_data_bottom20per['fc_wg_r'] = (
    sort_companies_data_bottom20per['r'] * sort_companies_data_bottom20per['fc_wg'])

"""
ret
"""
sort_companies_data_bottom20per['eq_wg_ret'] = (
    sort_companies_data_bottom20per['ret_ln'] * sort_companies_data_bottom20per['eq_wg'])
sort_companies_data_bottom20per['fc_wg_ret'] = (
    sort_companies_data_bottom20per['ret_ln'] * sort_companies_data_bottom20per['fc_wg'])

sort_companies_data_bottom20per.head(15)


,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,quantile_rank,eq_wg,sum_vol_180d,sum_ret,fc_wg,r,eq_wg_r,fc_wg_r,eq_wg_ret,fc_wg_ret
0,41,Hotel Properties Ltd,HPPS.SI,4,2017,NaN,NaN,NaN,NaN,NaN,...,0,0.076923,180.809394,-0.991675,0.048188,-0.087234,-0.006710,-0.004204,-0.005355,-0.003355
1,32,Great Eastern Holdings Ltd,GELA.SI,7,2017,32.771085,29.060755,34.766197,38.478261,1010.331945,...,0,0.076923,180.809394,-0.991675,0.050378,-0.124761,-0.009597,-0.006285,-0.008242,-0.005398
2,38,Hong Leong Finance Ltd,HLSF.SI,7,2017,NaN,NaN,NaN,NaN,NaN,...,0,0.076923,180.809394,-0.991675,0.062138,0.032294,0.002484,0.002007,0.003840,0.003102
3,71,Sheng Siong Group Ltd,SHEN.SI,5,2017,NaN,NaN,NaN,NaN,NaN,...,0,0.076923,180.809394,-0.991675,0.071226,0.118610,0.009124,0.008448,0.010479,0.009703
4,73,Singapore Airlines Ltd,SIAL.SI,3,2017,55.639475,59.621212,59.838752,45.077295,3567.658939,...,0,0.076923,180.809394,-0.991675,0.071226,-0.142221,-0.010940,-0.010130,-0.009585,-0.008875
5,55,Oversea-Chinese Banking Corporation Ltd,OCBC.SI,7,2017,45.455805,29.954202,41.070560,66.159420,1230.235824,...,0,0.076923,180.809394,-0.991675,0.071512,-0.113253,-0.008712,-0.008099,-0.007356,-0.006839
6,70,Singapore Exchange Ltd,SGXL.SI,7,2017,62.243342,41.574462,48.084723,85.228833,1999.096492,...,0,0.076923,180.809394,-0.991675,0.080348,-0.057378,-0.004414,-0.004610,-0.003058,-0.003195
7,19,DBS Group Holdings Ltd,DBSM.SI,7,2017,57.289860,62.386589,56.459314,72.333333,3522.304012,...,0,0.076923,180.809394,-0.991675,0.086308,-0.048955,-0.003766,-0.004225,-0.002410,-0.002704
8,84,United Overseas Bank Ltd,UOBH.SI,7,2017,64.067292,74.053823,62.708202,66.526570,4643.782093,...,0,0.076923,180.809394,-0.991675,0.090931,-0.091350,-0.007027,-0.008307,-0.005672,-0.006704
9,87,Venture Corporation Ltd,VENM.SI,8,2017,21.824429,4.618955,32.315622,32.826087,149.264390,...,0,0.076923,180.809394,-0.991675,0.091245,-0.401101,-0.030854,-0.036598,-0.029499,-0.034991


## Excess return portfolio
### Equal Weight and Factor Weight

In [2846]:
initial_year = 2017
stop_year = 2023

lo_eq_wg_r_p_dict = {}
lo_fc_wg_r_p_dict = {}
eq_wg_ret_p_dict = {}
fc_wg_ret_p_dict = {}
# company_dict2 = {}

#loop year, loop every months to get the companies quantile rank =4 and save it into company dictionary
for year in range(initial_year, stop_year+1, 1):

        each_year_companies = sort_companies_data_bottom20per.loc[(
            sort_companies_data_bottom20per['year'] == year)]

        """
        r
        """
        lo_eq_wg_r_p = each_year_companies['eq_wg_r'].sum()
        lo_fc_wg_r_p = each_year_companies['fc_wg_r'].sum()


        """
        ret
        """
        eq_wg_ret_p = each_year_companies['eq_wg_ret'].sum()
        fc_wg_ret_p = each_year_companies['fc_wg_ret'].sum()


        lo_eq_wg_r_p_dict.setdefault(year, lo_eq_wg_r_p)
        lo_fc_wg_r_p_dict.setdefault(year, lo_fc_wg_r_p)

        eq_wg_ret_p_dict.setdefault(year, eq_wg_ret_p)
        fc_wg_ret_p_dict.setdefault(year, fc_wg_ret_p)


print(f"lo_eq_wg_r_p_dict: {lo_eq_wg_r_p_dict}")
print(f"lo_fc_wg_r_p_dict: {lo_fc_wg_r_p_dict}")
print(f"eq_wg_ret_p_dict: {eq_wg_ret_p_dict}")
print(f"fc_wg_ret_p_dict: {fc_wg_ret_p_dict}")
# print(len(company_dict2))


lo_eq_wg_r_p_dict: {2017: -0.09390272991894873, 2018: 0.03803491391418538, 2019: -0.04585395409789429, 2020: 0.058092403840333526, 2021: 0.08882953243736927, 2022: 0.0, 2023: 0.0}
lo_fc_wg_r_p_dict: {2017: -0.10006993936547533, 2018: 0.0494866468468494, 2019: -0.03332659351615546, 2020: 0.059752796946947344, 2021: 0.06752280947830293, 2022: 0.0, 2023: 0.0}
eq_wg_ret_p_dict: {2017: -0.07628272991894873, 2018: 0.058164913914185386, 2019: -0.029283954097894305, 2020: 0.061142403840333544, 2021: 0.09324953243736926, 2022: -1.3000977131514846, 2023: 0.0}
fc_wg_ret_p_dict: {2017: -0.08244993936547532, 2018: 0.0696166468468494, 2019: -0.01675659351615546, 2020: 0.06280279694694736, 2021: 0.07194280947830295, 2022: -1.4094716052898824, 2023: 0.0}


In [2847]:
lo_eq_wg_r_p_dict_items = lo_eq_wg_r_p_dict.items()
lo_eq_wg_r_p_dict_list = list(lo_eq_wg_r_p_dict_items)

lo_eq_wg_r_p_dict_df = pd.DataFrame(lo_eq_wg_r_p_dict_list)
lo_eq_wg_r_p_dict_df.columns = ['year', 'lo_eq_wg_r_p']
lo_eq_wg_r_p_dict_df.replace(0, np.nan, inplace = True)
lo_eq_wg_r_p_dict_df.head()

,year,lo_eq_wg_r_p
0,2017,-0.093903
1,2018,0.038035
2,2019,-0.045854
3,2020,0.058092
4,2021,0.088830


In [2848]:
lo_fc_wg_r_p_dict_items = lo_fc_wg_r_p_dict.items()
lo_fc_wg_r_p_dict_list = list(lo_fc_wg_r_p_dict_items)

lo_fc_wg_r_p_dict_df = pd.DataFrame(lo_fc_wg_r_p_dict_list)
lo_fc_wg_r_p_dict_df.columns = ['year', 'lo_fc_wg_r_p']
lo_fc_wg_r_p_dict_df.replace(0, np.nan, inplace=True)
lo_fc_wg_r_p_dict_df.head()


,year,lo_fc_wg_r_p
0,2017,-0.100070
1,2018,0.049487
2,2019,-0.033327
3,2020,0.059753
4,2021,0.067523


In [2849]:
eq_wg_ret_p_dict_items = eq_wg_ret_p_dict.items()
eq_wg_ret_p_dict_list = list(eq_wg_ret_p_dict_items)

eq_wg_ret_p_dict_df = pd.DataFrame(eq_wg_ret_p_dict_list)
eq_wg_ret_p_dict_df.columns = ['year', 'eq_wg_ret_p']
eq_wg_ret_p_dict_df.replace(0, np.nan, inplace=True)
eq_wg_ret_p_dict_df.head()


,year,eq_wg_ret_p
0,2017,-0.076283
1,2018,0.058165
2,2019,-0.029284
3,2020,0.061142
4,2021,0.093250


In [2850]:
fc_wg_ret_p_dict_items = fc_wg_ret_p_dict.items()
fc_wg_ret_p_dict_list = list(fc_wg_ret_p_dict_items)

fc_wg_ret_p_dict_df = pd.DataFrame(fc_wg_ret_p_dict_list)
fc_wg_ret_p_dict_df.columns = ['year', 'fc_wg_ret_p']
fc_wg_ret_p_dict_df.replace(0, np.nan, inplace=True)
fc_wg_ret_p_dict_df.head()


,year,fc_wg_ret_p
0,2017,-0.082450
1,2018,0.069617
2,2019,-0.016757
3,2020,0.062803
4,2021,0.071943


In [2851]:
SG_esg_int = pd.merge(no_of_companies_all_df, no_of_companies_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, lo_eq_wg_r_p_dict_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, lo_fc_wg_r_p_dict_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, eq_wg_ret_p_dict_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, fc_wg_ret_p_dict_df,
                        on="year", how="left")
SG_esg_int.head()


,year,no of firms,no of firms(bottom 20),lo_eq_wg_r_p,lo_fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p
0,2017,52.0,13.0,-0.093903,-0.100070,-0.076283,-0.082450
1,2018,52.0,13.0,0.038035,0.049487,0.058165,0.069617
2,2019,52.0,13.0,-0.045854,-0.033327,-0.029284,-0.016757
3,2020,52.0,13.0,0.058092,0.059753,0.061142,0.062803
4,2021,52.0,13.0,0.088830,0.067523,0.093250,0.071943


### Merging the eq and fc wg of the r_p into the big dataframe

In [2852]:
sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    lo_eq_wg_r_p_dict_df, on='year')
sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    lo_fc_wg_r_p_dict_df, on='year')

sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    eq_wg_ret_p_dict_df, on='year')
sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    fc_wg_ret_p_dict_df, on='year')

sort_companies_data_bottom20per.head(15)


,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,fc_wg,r,eq_wg_r,fc_wg_r,eq_wg_ret,fc_wg_ret,lo_eq_wg_r_p,lo_fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p
0,41,Hotel Properties Ltd,HPPS.SI,4,2017,NaN,NaN,NaN,NaN,NaN,...,0.048188,-0.087234,-0.006710,-0.004204,-0.005355,-0.003355,-0.093903,-0.100070,-0.076283,-0.082450
1,32,Great Eastern Holdings Ltd,GELA.SI,7,2017,32.771085,29.060755,34.766197,38.478261,1010.331945,...,0.050378,-0.124761,-0.009597,-0.006285,-0.008242,-0.005398,-0.093903,-0.100070,-0.076283,-0.082450
2,38,Hong Leong Finance Ltd,HLSF.SI,7,2017,NaN,NaN,NaN,NaN,NaN,...,0.062138,0.032294,0.002484,0.002007,0.003840,0.003102,-0.093903,-0.100070,-0.076283,-0.082450
3,71,Sheng Siong Group Ltd,SHEN.SI,5,2017,NaN,NaN,NaN,NaN,NaN,...,0.071226,0.118610,0.009124,0.008448,0.010479,0.009703,-0.093903,-0.100070,-0.076283,-0.082450
4,73,Singapore Airlines Ltd,SIAL.SI,3,2017,55.639475,59.621212,59.838752,45.077295,3567.658939,...,0.071226,-0.142221,-0.010940,-0.010130,-0.009585,-0.008875,-0.093903,-0.100070,-0.076283,-0.082450
5,55,Oversea-Chinese Banking Corporation Ltd,OCBC.SI,7,2017,45.455805,29.954202,41.070560,66.159420,1230.235824,...,0.071512,-0.113253,-0.008712,-0.008099,-0.007356,-0.006839,-0.093903,-0.100070,-0.076283,-0.082450
6,70,Singapore Exchange Ltd,SGXL.SI,7,2017,62.243342,41.574462,48.084723,85.228833,1999.096492,...,0.080348,-0.057378,-0.004414,-0.004610,-0.003058,-0.003195,-0.093903,-0.100070,-0.076283,-0.082450
7,19,DBS Group Holdings Ltd,DBSM.SI,7,2017,57.289860,62.386589,56.459314,72.333333,3522.304012,...,0.086308,-0.048955,-0.003766,-0.004225,-0.002410,-0.002704,-0.093903,-0.100070,-0.076283,-0.082450
8,84,United Overseas Bank Ltd,UOBH.SI,7,2017,64.067292,74.053823,62.708202,66.526570,4643.782093,...,0.090931,-0.091350,-0.007027,-0.008307,-0.005672,-0.006704,-0.093903,-0.100070,-0.076283,-0.082450
9,87,Venture Corporation Ltd,VENM.SI,8,2017,21.824429,4.618955,32.315622,32.826087,149.264390,...,0.091245,-0.401101,-0.030854,-0.036598,-0.029499,-0.034991,-0.093903,-0.100070,-0.076283,-0.082450


In [2853]:
sort_companies_data_bottom20per['rm_r_y1'] = (
    sort_companies_data_bottom20per['rm'] - sort_companies_data_bottom20per['rfr']) * 12
sort_companies_data_bottom20per['rm_r_y2'] = (((1+(
    sort_companies_data_bottom20per['rm'] - sort_companies_data_bottom20per['rfr'])) ** 12) - 1)

sort_companies_data_bottom20per['lo_eq_wg_r_p_y1'] = sort_companies_data_bottom20per['lo_eq_wg_r_p'] *12
sort_companies_data_bottom20per['lo_eq_wg_r_p_y2'] = (((1 + sort_companies_data_bottom20per['lo_eq_wg_r_p'])**12) - 1)
sort_companies_data_bottom20per['eq_r_alpha_y1'] = sort_companies_data_bottom20per['lo_eq_wg_r_p_y1'] - \
    sort_companies_data_bottom20per['rm_r_y1']
sort_companies_data_bottom20per['eq_r_alpha_y2'] = sort_companies_data_bottom20per['lo_eq_wg_r_p_y2'] - \
    sort_companies_data_bottom20per['rm_r_y2']
sort_companies_data_bottom20per.head()


,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,lo_eq_wg_r_p,lo_fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p,rm_r_y1,rm_r_y2,lo_eq_wg_r_p_y1,lo_eq_wg_r_p_y2,eq_r_alpha_y1,eq_r_alpha_y2
0,41,Hotel Properties Ltd,HPPS.SI,4,2017,NaN,NaN,NaN,NaN,NaN,...,-0.093903,-0.10007,-0.076283,-0.08245,1.191128,2.113208,-1.126833,-0.693735,-2.317961,-2.806943
1,32,Great Eastern Holdings Ltd,GELA.SI,7,2017,32.771085,29.060755,34.766197,38.478261,1010.331945,...,-0.093903,-0.10007,-0.076283,-0.08245,1.191128,2.113208,-1.126833,-0.693735,-2.317961,-2.806943
2,38,Hong Leong Finance Ltd,HLSF.SI,7,2017,NaN,NaN,NaN,NaN,NaN,...,-0.093903,-0.10007,-0.076283,-0.08245,1.191128,2.113208,-1.126833,-0.693735,-2.317961,-2.806943
3,71,Sheng Siong Group Ltd,SHEN.SI,5,2017,NaN,NaN,NaN,NaN,NaN,...,-0.093903,-0.10007,-0.076283,-0.08245,1.191128,2.113208,-1.126833,-0.693735,-2.317961,-2.806943
4,73,Singapore Airlines Ltd,SIAL.SI,3,2017,55.639475,59.621212,59.838752,45.077295,3567.658939,...,-0.093903,-0.10007,-0.076283,-0.08245,1.191128,2.113208,-1.126833,-0.693735,-2.317961,-2.806943


In [2854]:
sort_companies_data_bottom20per['lo_fc_wg_r_p_y1'] = sort_companies_data_bottom20per['lo_fc_wg_r_p'] * 12
sort_companies_data_bottom20per['lo_fc_wg_r_p_y2'] = ((
    (1 + sort_companies_data_bottom20per['lo_fc_wg_r_p'])**12) - 1)
sort_companies_data_bottom20per['fc_r_alpha_y1'] = sort_companies_data_bottom20per['lo_fc_wg_r_p_y1'] - \
    sort_companies_data_bottom20per['rm_r_y1']
sort_companies_data_bottom20per['fc_r_alpha_y2'] = sort_companies_data_bottom20per['lo_fc_wg_r_p_y2'] - \
    sort_companies_data_bottom20per['rm_r_y2']
sort_companies_data_bottom20per.head()


,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,rm_r_y1,rm_r_y2,lo_eq_wg_r_p_y1,lo_eq_wg_r_p_y2,eq_r_alpha_y1,eq_r_alpha_y2,lo_fc_wg_r_p_y1,lo_fc_wg_r_p_y2,fc_r_alpha_y1,fc_r_alpha_y2
0,41,Hotel Properties Ltd,HPPS.SI,4,2017,NaN,NaN,NaN,NaN,NaN,...,1.191128,2.113208,-1.126833,-0.693735,-2.317961,-2.806943,-1.200839,-0.717834,-2.391967,-2.831042
1,32,Great Eastern Holdings Ltd,GELA.SI,7,2017,32.771085,29.060755,34.766197,38.478261,1010.331945,...,1.191128,2.113208,-1.126833,-0.693735,-2.317961,-2.806943,-1.200839,-0.717834,-2.391967,-2.831042
2,38,Hong Leong Finance Ltd,HLSF.SI,7,2017,NaN,NaN,NaN,NaN,NaN,...,1.191128,2.113208,-1.126833,-0.693735,-2.317961,-2.806943,-1.200839,-0.717834,-2.391967,-2.831042
3,71,Sheng Siong Group Ltd,SHEN.SI,5,2017,NaN,NaN,NaN,NaN,NaN,...,1.191128,2.113208,-1.126833,-0.693735,-2.317961,-2.806943,-1.200839,-0.717834,-2.391967,-2.831042
4,73,Singapore Airlines Ltd,SIAL.SI,3,2017,55.639475,59.621212,59.838752,45.077295,3567.658939,...,1.191128,2.113208,-1.126833,-0.693735,-2.317961,-2.806943,-1.200839,-0.717834,-2.391967,-2.831042


In [2855]:

# sort_companies_data_bottom20per.to_excel(
#     "../3.04/SG_esg_bottom20.xlsx", sheet_name="main")

## CAPM-adjusted return ($r_{c}$)

$r_(c,t)=r_(f,t)+β_(p,t) (r_(m,t)-r_(f,t))$

In [2856]:
sort_companies_data_bottom20per['rc'] = sort_companies_data_bottom20per['rfr'] + \
    (sort_companies_data_bottom20per['beta_5y'] *
    (sort_companies_data_bottom20per['rm'] - sort_companies_data_bottom20per['rfr']))
    
"""
equal weight rc
"""
sort_companies_data_bottom20per['lo_eq_wg_rc_int'] = sort_companies_data_bottom20per ['eq_wg'] * sort_companies_data_bottom20per['rc']

"""
factor weight rc
"""
sort_companies_data_bottom20per['lo_fc_wg_rc_int'] = sort_companies_data_bottom20per['fc_wg'] * \
    sort_companies_data_bottom20per['rc']

sort_companies_data_bottom20per.head()


,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,lo_eq_wg_r_p_y2,eq_r_alpha_y1,eq_r_alpha_y2,lo_fc_wg_r_p_y1,lo_fc_wg_r_p_y2,fc_r_alpha_y1,fc_r_alpha_y2,rc,lo_eq_wg_rc_int,lo_fc_wg_rc_int
0,41,Hotel Properties Ltd,HPPS.SI,4,2017,NaN,NaN,NaN,NaN,NaN,...,-0.693735,-2.317961,-2.806943,-1.200839,-0.717834,-2.391967,-2.831042,0.087753,0.006750,0.004229
1,32,Great Eastern Holdings Ltd,GELA.SI,7,2017,32.771085,29.060755,34.766197,38.478261,1010.331945,...,-0.693735,-2.317961,-2.806943,-1.200839,-0.717834,-2.391967,-2.831042,0.083689,0.006438,0.004216
2,38,Hong Leong Finance Ltd,HLSF.SI,7,2017,NaN,NaN,NaN,NaN,NaN,...,-0.693735,-2.317961,-2.806943,-1.200839,-0.717834,-2.391967,-2.831042,0.081136,0.006241,0.005042
3,71,Sheng Siong Group Ltd,SHEN.SI,5,2017,NaN,NaN,NaN,NaN,NaN,...,-0.693735,-2.317961,-2.806943,-1.200839,-0.717834,-2.391967,-2.831042,0.028537,0.002195,0.002033
4,73,Singapore Airlines Ltd,SIAL.SI,3,2017,55.639475,59.621212,59.838752,45.077295,3567.658939,...,-0.693735,-2.317961,-2.806943,-1.200839,-0.717834,-2.391967,-2.831042,0.135719,0.010440,0.009667


In [2857]:
initial_year = 2017
stop_year = 2023

lo_eq_wg_rc_p_dict = {}
lo_fc_wg_rc_p_dict = {}

# company_dict2 = {}

#loop year, loop every months to get the companies quantile rank =4 and save it into company dictionary
for year in range(initial_year, stop_year+1, 1):

        each_year_companies = sort_companies_data_bottom20per.loc[(
            sort_companies_data_bottom20per['year'] == year)]

        """
        rc
        """
        lo_eq_wg_rc_p = each_year_companies['lo_eq_wg_rc_int'].sum()
        lo_fc_wg_rc_p = each_year_companies['lo_fc_wg_rc_int'].sum()

        lo_eq_wg_rc_p_dict.setdefault(year, lo_eq_wg_rc_p)
        lo_fc_wg_rc_p_dict.setdefault(year, lo_fc_wg_rc_p)


print(f"lo_eq_wg_rc_p_dict: {lo_eq_wg_rc_p_dict}")
print(f"lo_fc_wg_rc_p_dict: {lo_fc_wg_rc_p_dict}")


lo_eq_wg_rc_p_dict: {2017: 0.09242869976037921, 2018: -0.09425596042482937, 2019: 0.011797803182118396, 2020: -0.07331228551282716, 2021: 0.08307333045347635, 2022: 0.0, 2023: 0.0}
lo_fc_wg_rc_p_dict: {2017: 0.09300692100869666, 2018: -0.09514008744533221, 2019: 0.011760917290369296, 2020: -0.07390251494062142, 2021: 0.08368126812736733, 2022: 0.0, 2023: 0.0}


In [2858]:
lo_eq_wg_rc_p_dict_items = lo_eq_wg_rc_p_dict.items()
lo_eq_wg_rc_p_dict_list = list(lo_eq_wg_rc_p_dict_items)

lo_eq_wg_rc_p_dict_df = pd.DataFrame(lo_eq_wg_rc_p_dict_list)
lo_eq_wg_rc_p_dict_df.columns = ['year', 'lo_eq_wg_rc_p']
lo_eq_wg_rc_p_dict_df.replace(0, np.nan, inplace=True)
lo_eq_wg_rc_p_dict_df.head()


,year,lo_eq_wg_rc_p
0,2017,0.092429
1,2018,-0.094256
2,2019,0.011798
3,2020,-0.073312
4,2021,0.083073


In [2859]:
lo_fc_wg_rc_p_dict_items = lo_fc_wg_rc_p_dict.items()
lo_fc_wg_rc_p_dict_list = list(lo_fc_wg_rc_p_dict_items)

lo_fc_wg_rc_p_dict_df = pd.DataFrame(lo_fc_wg_rc_p_dict_list).dropna()
lo_fc_wg_rc_p_dict_df.columns = ['year', 'lo_fc_wg_rc_p']
lo_fc_wg_rc_p_dict_df.replace(0, np.nan, inplace=True)
lo_fc_wg_rc_p_dict_df.head()


,year,lo_fc_wg_rc_p
0,2017,0.093007
1,2018,-0.095140
2,2019,0.011761
3,2020,-0.073903
4,2021,0.083681


In [2860]:
SG_esg_int = pd.merge(SG_esg_int, lo_eq_wg_rc_p_dict_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, lo_fc_wg_rc_p_dict_df,
                        on="year", how="left")
SG_esg_int.head()

,year,no of firms,no of firms(bottom 20),lo_eq_wg_r_p,lo_fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p,lo_eq_wg_rc_p,lo_fc_wg_rc_p
0,2017,52.0,13.0,-0.093903,-0.100070,-0.076283,-0.082450,0.092429,0.093007
1,2018,52.0,13.0,0.038035,0.049487,0.058165,0.069617,-0.094256,-0.095140
2,2019,52.0,13.0,-0.045854,-0.033327,-0.029284,-0.016757,0.011798,0.011761
3,2020,52.0,13.0,0.058092,0.059753,0.061142,0.062803,-0.073312,-0.073903
4,2021,52.0,13.0,0.088830,0.067523,0.093250,0.071943,0.083073,0.083681


In [2861]:
sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    lo_eq_wg_rc_p_dict_df, on='year')
sort_companies_data_bottom20per = sort_companies_data_bottom20per.merge(
    lo_fc_wg_rc_p_dict_df, on='year')

sort_companies_data_bottom20per.head(15)

,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,eq_r_alpha_y2,lo_fc_wg_r_p_y1,lo_fc_wg_r_p_y2,fc_r_alpha_y1,fc_r_alpha_y2,rc,lo_eq_wg_rc_int,lo_fc_wg_rc_int,lo_eq_wg_rc_p,lo_fc_wg_rc_p
0,41,Hotel Properties Ltd,HPPS.SI,4,2017,NaN,NaN,NaN,NaN,NaN,...,-2.806943,-1.200839,-0.717834,-2.391967,-2.831042,0.087753,0.006750,0.004229,0.092429,0.093007
1,32,Great Eastern Holdings Ltd,GELA.SI,7,2017,32.771085,29.060755,34.766197,38.478261,1010.331945,...,-2.806943,-1.200839,-0.717834,-2.391967,-2.831042,0.083689,0.006438,0.004216,0.092429,0.093007
2,38,Hong Leong Finance Ltd,HLSF.SI,7,2017,NaN,NaN,NaN,NaN,NaN,...,-2.806943,-1.200839,-0.717834,-2.391967,-2.831042,0.081136,0.006241,0.005042,0.092429,0.093007
3,71,Sheng Siong Group Ltd,SHEN.SI,5,2017,NaN,NaN,NaN,NaN,NaN,...,-2.806943,-1.200839,-0.717834,-2.391967,-2.831042,0.028537,0.002195,0.002033,0.092429,0.093007
4,73,Singapore Airlines Ltd,SIAL.SI,3,2017,55.639475,59.621212,59.838752,45.077295,3567.658939,...,-2.806943,-1.200839,-0.717834,-2.391967,-2.831042,0.135719,0.010440,0.009667,0.092429,0.093007
5,55,Oversea-Chinese Banking Corporation Ltd,OCBC.SI,7,2017,45.455805,29.954202,41.070560,66.159420,1230.235824,...,-2.806943,-1.200839,-0.717834,-2.391967,-2.831042,0.131198,0.010092,0.009382,0.092429,0.093007
6,70,Singapore Exchange Ltd,SGXL.SI,7,2017,62.243342,41.574462,48.084723,85.228833,1999.096492,...,-2.806943,-1.200839,-0.717834,-2.391967,-2.831042,0.056543,0.004349,0.004543,0.092429,0.093007
7,19,DBS Group Holdings Ltd,DBSM.SI,7,2017,57.289860,62.386589,56.459314,72.333333,3522.304012,...,-2.806943,-1.200839,-0.717834,-2.391967,-2.831042,0.152073,0.011698,0.013125,0.092429,0.093007
8,84,United Overseas Bank Ltd,UOBH.SI,7,2017,64.067292,74.053823,62.708202,66.526570,4643.782093,...,-2.806943,-1.200839,-0.717834,-2.391967,-2.831042,0.140256,0.010789,0.012754,0.092429,0.093007
9,87,Venture Corporation Ltd,VENM.SI,8,2017,21.824429,4.618955,32.315622,32.826087,149.264390,...,-2.806943,-1.200839,-0.717834,-2.391967,-2.831042,0.076988,0.005922,0.007025,0.092429,0.093007


In [2862]:
sort_companies_data_bottom20per['lo_eq_wg_rc_p_y1'] = sort_companies_data_bottom20per['lo_eq_wg_rc_p'] * 12
sort_companies_data_bottom20per['eq_wg_rc_p_y2'] = ((
    (1 + sort_companies_data_bottom20per['lo_eq_wg_rc_p'])**12) - 1)
sort_companies_data_bottom20per['eq_rc_alpha_y1'] = sort_companies_data_bottom20per['lo_eq_wg_rc_p_y1'] - \
    sort_companies_data_bottom20per['rm_r_y1']
sort_companies_data_bottom20per['eq_rc_alpha_y2'] = sort_companies_data_bottom20per['eq_wg_rc_p_y2'] - \
    sort_companies_data_bottom20per['rm_r_y2']
sort_companies_data_bottom20per.head()


,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,fc_r_alpha_y2,rc,lo_eq_wg_rc_int,lo_fc_wg_rc_int,lo_eq_wg_rc_p,lo_fc_wg_rc_p,lo_eq_wg_rc_p_y1,eq_wg_rc_p_y2,eq_rc_alpha_y1,eq_rc_alpha_y2
0,41,Hotel Properties Ltd,HPPS.SI,4,2017,NaN,NaN,NaN,NaN,NaN,...,-2.831042,0.087753,0.006750,0.004229,0.092429,0.093007,1.109144,1.888798,-0.081983,-0.22441
1,32,Great Eastern Holdings Ltd,GELA.SI,7,2017,32.771085,29.060755,34.766197,38.478261,1010.331945,...,-2.831042,0.083689,0.006438,0.004216,0.092429,0.093007,1.109144,1.888798,-0.081983,-0.22441
2,38,Hong Leong Finance Ltd,HLSF.SI,7,2017,NaN,NaN,NaN,NaN,NaN,...,-2.831042,0.081136,0.006241,0.005042,0.092429,0.093007,1.109144,1.888798,-0.081983,-0.22441
3,71,Sheng Siong Group Ltd,SHEN.SI,5,2017,NaN,NaN,NaN,NaN,NaN,...,-2.831042,0.028537,0.002195,0.002033,0.092429,0.093007,1.109144,1.888798,-0.081983,-0.22441
4,73,Singapore Airlines Ltd,SIAL.SI,3,2017,55.639475,59.621212,59.838752,45.077295,3567.658939,...,-2.831042,0.135719,0.010440,0.009667,0.092429,0.093007,1.109144,1.888798,-0.081983,-0.22441


In [2863]:
sort_companies_data_bottom20per['lo_fc_wg_rc_p_y1'] = sort_companies_data_bottom20per['lo_fc_wg_rc_p'] * 12
sort_companies_data_bottom20per['lo_fc_wg_rc_p_y2'] = ((
    (1 + sort_companies_data_bottom20per['lo_fc_wg_rc_p'])**12) - 1)
sort_companies_data_bottom20per['fc_rc_alpha_y1'] = sort_companies_data_bottom20per['lo_fc_wg_rc_p_y1'] - \
    sort_companies_data_bottom20per['rm_r_y1']
sort_companies_data_bottom20per['fc_rc_alpha_y2'] = sort_companies_data_bottom20per['lo_fc_wg_rc_p_y2'] - \
    sort_companies_data_bottom20per['rm_r_y2']
sort_companies_data_bottom20per.head()


,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,lo_eq_wg_rc_p,lo_fc_wg_rc_p,lo_eq_wg_rc_p_y1,eq_wg_rc_p_y2,eq_rc_alpha_y1,eq_rc_alpha_y2,lo_fc_wg_rc_p_y1,lo_fc_wg_rc_p_y2,fc_rc_alpha_y1,fc_rc_alpha_y2
0,41,Hotel Properties Ltd,HPPS.SI,4,2017,NaN,NaN,NaN,NaN,NaN,...,0.092429,0.093007,1.109144,1.888798,-0.081983,-0.22441,1.116083,1.9072,-0.075045,-0.206008
1,32,Great Eastern Holdings Ltd,GELA.SI,7,2017,32.771085,29.060755,34.766197,38.478261,1010.331945,...,0.092429,0.093007,1.109144,1.888798,-0.081983,-0.22441,1.116083,1.9072,-0.075045,-0.206008
2,38,Hong Leong Finance Ltd,HLSF.SI,7,2017,NaN,NaN,NaN,NaN,NaN,...,0.092429,0.093007,1.109144,1.888798,-0.081983,-0.22441,1.116083,1.9072,-0.075045,-0.206008
3,71,Sheng Siong Group Ltd,SHEN.SI,5,2017,NaN,NaN,NaN,NaN,NaN,...,0.092429,0.093007,1.109144,1.888798,-0.081983,-0.22441,1.116083,1.9072,-0.075045,-0.206008
4,73,Singapore Airlines Ltd,SIAL.SI,3,2017,55.639475,59.621212,59.838752,45.077295,3567.658939,...,0.092429,0.093007,1.109144,1.888798,-0.081983,-0.22441,1.116083,1.9072,-0.075045,-0.206008


In [2864]:
SG_int_wanted = sort_companies_data_bottom20per[[
    'year', 'rm_r_y1', 'rm_r_y2', 'lo_eq_wg_r_p_y1', 'eq_r_alpha_y1', 'eq_r_alpha_y2', 'lo_eq_wg_r_p_y2', 'lo_fc_wg_r_p_y1', 'fc_r_alpha_y1', 'fc_r_alpha_y2', 'lo_fc_wg_r_p_y2', 'lo_eq_wg_rc_p_y1', 'eq_rc_alpha_y1', 'eq_rc_alpha_y2', 'eq_wg_rc_p_y2', 'lo_fc_wg_rc_p_y1', 'fc_rc_alpha_y1', 'fc_rc_alpha_y2', 'lo_fc_wg_rc_p_y2']]
SG_int_wanted = SG_int_wanted.drop_duplicates()
SG_int_wanted.head()


,year,rm_r_y1,rm_r_y2,lo_eq_wg_r_p_y1,eq_r_alpha_y1,eq_r_alpha_y2,lo_eq_wg_r_p_y2,lo_fc_wg_r_p_y1,fc_r_alpha_y1,fc_r_alpha_y2,lo_fc_wg_r_p_y2,lo_eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_rc_alpha_y2,eq_wg_rc_p_y2,lo_fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_rc_alpha_y2,lo_fc_wg_rc_p_y2
0,2017,1.191128,2.113208,-1.126833,-2.317961,-2.806943,-0.693735,-1.200839,-2.391967,-2.831042,-0.717834,1.109144,-0.081983,-0.224410,1.888798,1.116083,-0.075045,-0.206008,1.907200
13,2018,-1.821289,-0.861280,0.456419,2.277708,1.426385,0.565105,0.593840,2.415129,1.646629,0.785349,-1.131072,0.690218,0.166116,-0.695164,-1.141681,0.679608,0.162564,-0.698716
26,2019,-0.075984,-0.073393,-0.550247,-0.474263,-0.357258,-0.430651,-0.399919,-0.323935,-0.260786,-0.334179,0.141574,0.217558,0.224524,0.151131,0.141131,0.217115,0.224021,0.150628
39,2020,-1.215864,-0.722509,0.697109,1.912973,1.691679,0.969170,0.717034,1.932898,1.729081,1.006573,-0.879747,0.336117,0.123561,-0.598948,-0.886830,0.329034,0.120506,-0.602002
52,2021,1.252343,2.291070,1.065954,-0.186389,-0.514436,1.776634,0.810274,-0.442069,-1.100657,1.190413,0.996880,-0.255463,-0.685551,1.605520,1.004175,-0.248168,-0.667946,1.623124


In [2865]:
SG_esg_int = pd.merge(SG_esg_int, SG_int_wanted,
                        on="year", how="left")
SG_esg_int.head()


,year,no of firms,no of firms(bottom 20),lo_eq_wg_r_p,lo_fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p,lo_eq_wg_rc_p,lo_fc_wg_rc_p,rm_r_y1,...,fc_r_alpha_y2,lo_fc_wg_r_p_y2,lo_eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_rc_alpha_y2,eq_wg_rc_p_y2,lo_fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_rc_alpha_y2,lo_fc_wg_rc_p_y2
0,2017,52.0,13.0,-0.093903,-0.100070,-0.076283,-0.082450,0.092429,0.093007,1.191128,...,-2.831042,-0.717834,1.109144,-0.081983,-0.224410,1.888798,1.116083,-0.075045,-0.206008,1.907200
1,2018,52.0,13.0,0.038035,0.049487,0.058165,0.069617,-0.094256,-0.095140,-1.821289,...,1.646629,0.785349,-1.131072,0.690218,0.166116,-0.695164,-1.141681,0.679608,0.162564,-0.698716
2,2019,52.0,13.0,-0.045854,-0.033327,-0.029284,-0.016757,0.011798,0.011761,-0.075984,...,-0.260786,-0.334179,0.141574,0.217558,0.224524,0.151131,0.141131,0.217115,0.224021,0.150628
3,2020,52.0,13.0,0.058092,0.059753,0.061142,0.062803,-0.073312,-0.073903,-1.215864,...,1.729081,1.006573,-0.879747,0.336117,0.123561,-0.598948,-0.886830,0.329034,0.120506,-0.602002
4,2021,52.0,13.0,0.088830,0.067523,0.093250,0.071943,0.083073,0.083681,1.252343,...,-1.100657,1.190413,0.996880,-0.255463,-0.685551,1.605520,1.004175,-0.248168,-0.667946,1.623124


In [2866]:
SG_esg_int = SG_esg_int[['year', 'no of firms', 'no of firms(bottom 20)',
                            'rm_r_y1', 'rm_r_y2', 'eq_wg_ret_p', 'fc_wg_ret_p', 'lo_eq_wg_r_p',
                            'lo_eq_wg_r_p_y1', 'eq_r_alpha_y1', 'lo_eq_wg_r_p_y2', 'eq_r_alpha_y2', 'lo_fc_wg_r_p',
                            'lo_fc_wg_r_p_y1', 'fc_r_alpha_y1', 'lo_fc_wg_r_p_y2', 'fc_r_alpha_y2', 'lo_eq_wg_rc_p',
                            'lo_eq_wg_rc_p_y1', 'eq_rc_alpha_y1', 'eq_wg_rc_p_y2', 'eq_rc_alpha_y2', 'lo_fc_wg_rc_p',
                            'lo_fc_wg_rc_p_y1', 'fc_rc_alpha_y1', 'lo_fc_wg_rc_p_y2', 'fc_rc_alpha_y2']]
SG_esg_int.head()


,year,no of firms,no of firms(bottom 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,lo_eq_wg_r_p,lo_eq_wg_r_p_y1,eq_r_alpha_y1,...,lo_eq_wg_rc_p,lo_eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_wg_rc_p_y2,eq_rc_alpha_y2,lo_fc_wg_rc_p,lo_fc_wg_rc_p_y1,fc_rc_alpha_y1,lo_fc_wg_rc_p_y2,fc_rc_alpha_y2
0,2017,52.0,13.0,1.191128,2.113208,-0.076283,-0.082450,-0.093903,-1.126833,-2.317961,...,0.092429,1.109144,-0.081983,1.888798,-0.224410,0.093007,1.116083,-0.075045,1.907200,-0.206008
1,2018,52.0,13.0,-1.821289,-0.861280,0.058165,0.069617,0.038035,0.456419,2.277708,...,-0.094256,-1.131072,0.690218,-0.695164,0.166116,-0.095140,-1.141681,0.679608,-0.698716,0.162564
2,2019,52.0,13.0,-0.075984,-0.073393,-0.029284,-0.016757,-0.045854,-0.550247,-0.474263,...,0.011798,0.141574,0.217558,0.151131,0.224524,0.011761,0.141131,0.217115,0.150628,0.224021
3,2020,52.0,13.0,-1.215864,-0.722509,0.061142,0.062803,0.058092,0.697109,1.912973,...,-0.073312,-0.879747,0.336117,-0.598948,0.123561,-0.073903,-0.886830,0.329034,-0.602002,0.120506
4,2021,52.0,13.0,1.252343,2.291070,0.093250,0.071943,0.088830,1.065954,-0.186389,...,0.083073,0.996880,-0.255463,1.605520,-0.685551,0.083681,1.004175,-0.248168,1.623124,-0.667946


## Average of the int_reporting

In [2867]:
SG_esg_int['year'] = SG_esg_int['year'].astype(float)
SG_esg_int_mean = SG_esg_int.mean(numeric_only=True)
SG_esg_int_mean = pd.DataFrame(SG_esg_int_mean).T
SG_esg_int_mean['year'] = SG_esg_int_mean['year'].replace([SG_esg_int_mean['year'].mean()], 'average')
SG_esg_int_mean = pd.concat([SG_esg_int, SG_esg_int_mean], axis=0)
SG_esg_int_mean.tail()

,year,no of firms,no of firms(bottom 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,lo_eq_wg_r_p,lo_eq_wg_r_p_y1,eq_r_alpha_y1,...,lo_eq_wg_rc_p,lo_eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_wg_rc_p_y2,eq_rc_alpha_y2,lo_fc_wg_rc_p,lo_fc_wg_rc_p_y1,fc_rc_alpha_y1,lo_fc_wg_rc_p_y2,fc_rc_alpha_y2
2,2019.0,52.0,13.000000,-0.075984,-0.073393,-0.029284,-0.016757,-0.045854,-0.550247,-0.474263,...,0.011798,0.141574,0.217558,0.151131,0.224524,0.011761,0.141131,0.217115,0.150628,0.224021
3,2020.0,52.0,13.000000,-1.215864,-0.722509,0.061142,0.062803,0.058092,0.697109,1.912973,...,-0.073312,-0.879747,0.336117,-0.598948,0.123561,-0.073903,-0.886830,0.329034,-0.602002,0.120506
4,2021.0,52.0,13.000000,1.252343,2.291070,0.093250,0.071943,0.088830,1.065954,-0.186389,...,0.083073,0.996880,-0.255463,1.605520,-0.685551,0.083681,1.004175,-0.248168,1.623124,-0.667946
5,2022.0,34.0,9.000000,NaN,NaN,-1.300098,-1.409472,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,average,49.0,12.333333,-0.133933,0.549419,-0.198851,-0.217386,0.009040,0.108480,0.242414,...,0.003946,0.047356,0.181289,0.470267,-0.079152,0.003881,0.046576,0.180509,0.476047,-0.073373


## T-test of the int_reporting

a: an array of sample observations <br>
popmean: the expected population mean (The population mean is an average of a group characteristic.) <br>

In [2868]:
SG_res_dict = {}
temp_list = []

"""
make the t-stat, p-value into dictionary
"""
for column in range(3, SG_esg_int.columns.__len__()):
    tscore, pvalue = ttest_1samp(
        a=SG_esg_int.dropna()[SG_esg_int.columns[column]], popmean=0)
    temp_dict = {SG_esg_int.columns[column]: {
        "t-score": tscore, "p-value": pvalue}}
    SG_res_dict = {**SG_res_dict, **temp_dict}

"""
make the dictionary into dataframe
"""
res_df = pd.DataFrame.from_dict({(i): SG_res_dict[i]
                                    for i in SG_res_dict.keys()},
                                orient='index').T
res_df.reset_index(inplace=True)
res_df.rename(columns={'index': 'year'}, inplace=True)
res_df
# SG_res_dict = {**temp_dict, **SG_res_dict}
# testing_df = pd.DataFrame(SG_res_dict, index=[0])
# testing_df
# SG_res_dict

,year,rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,lo_eq_wg_r_p,lo_eq_wg_r_p_y1,eq_r_alpha_y1,lo_eq_wg_r_p_y2,eq_r_alpha_y2,...,lo_eq_wg_rc_p,lo_eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_wg_rc_p_y2,eq_rc_alpha_y2,lo_fc_wg_rc_p,lo_fc_wg_rc_p_y1,fc_rc_alpha_y1,lo_fc_wg_rc_p_y2,fc_rc_alpha_y2
0,t-score,-0.215872,0.798251,0.673236,0.685347,0.265289,0.265289,0.287832,0.962841,-0.138504,...,0.102296,0.102296,1.098864,0.865515,-0.463882,0.099834,0.099834,1.117782,0.868260,-0.441328
1,p-value,0.839649,0.469433,0.537693,0.530779,0.803898,0.803898,0.787773,0.390152,0.896535,...,0.923445,0.923445,0.333524,0.435580,0.666853,0.925279,0.925279,0.326260,0.434241,0.681782


In [2869]:
SG_esg_int = pd.concat([SG_esg_int_mean, res_df])
SG_esg_int.tail()

,year,no of firms,no of firms(bottom 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,lo_eq_wg_r_p,lo_eq_wg_r_p_y1,eq_r_alpha_y1,...,lo_eq_wg_rc_p,lo_eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_wg_rc_p_y2,eq_rc_alpha_y2,lo_fc_wg_rc_p,lo_fc_wg_rc_p_y1,fc_rc_alpha_y1,lo_fc_wg_rc_p_y2,fc_rc_alpha_y2
4,2021.0,52.0,13.000000,1.252343,2.291070,0.093250,0.071943,0.088830,1.065954,-0.186389,...,0.083073,0.996880,-0.255463,1.605520,-0.685551,0.083681,1.004175,-0.248168,1.623124,-0.667946
5,2022.0,34.0,9.000000,NaN,NaN,-1.300098,-1.409472,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,average,49.0,12.333333,-0.133933,0.549419,-0.198851,-0.217386,0.009040,0.108480,0.242414,...,0.003946,0.047356,0.181289,0.470267,-0.079152,0.003881,0.046576,0.180509,0.476047,-0.073373
0,t-score,NaN,NaN,-0.215872,0.798251,0.673236,0.685347,0.265289,0.265289,0.287832,...,0.102296,0.102296,1.098864,0.865515,-0.463882,0.099834,0.099834,1.117782,0.868260,-0.441328
1,p-value,NaN,NaN,0.839649,0.469433,0.537693,0.530779,0.803898,0.803898,0.787773,...,0.923445,0.923445,0.333524,0.435580,0.666853,0.925279,0.925279,0.326260,0.434241,0.681782


### Export the result into excel file

In [2870]:
sort_companies_data_bottom20per.to_excel(bottom_path, sheet_name="main", index=False)
SG_esg_int.to_excel(reporting_path, sheet_name="main", index=False)
